In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os
import json
import random
import argparse
from PIL import Image
import pandas as pd

# Caption templates
TEMPLATES = [
    "A {style} {room} interior with {feature}.",
    "A photorealistic {style} style {room} featuring {feature}.",
    "A {style} {room} with {feature} and elegant design.",
    "{style} {room} interior with {feature} and natural lighting."
]

FEATURES = [
    "warm lighting", "wooden textures", "white walls", "minimalist decor",
    "open layout", "cozy atmosphere", "natural tones", "modern furniture",
    "marble flooring", "industrial elements"
]

def create_caption(style, room):
    return random.choice(TEMPLATES).format(
        style=style,
        room=room,
        feature=random.choice(FEATURES)
    )

def process_row(row, base_dir, output_dir, size, idx):
    room = row["room_type"]
    style = row["style"]

    # Actual folder: /drive/.../kitchen/modern/*.jpg
    folder = os.path.join(base_dir, room, style)

    # Try to find matching image
    # We DO NOT trust CSV image_path
    files = os.listdir(folder)

    # Choose a random image from the folder
    img_name = random.choice(files)
    in_path = os.path.join(folder, img_name)

    # Output resized image
    out_name = f"{style}_{room}_{idx:06d}.jpg"
    out_path = os.path.join(output_dir, "images", out_name)

    # Create output folder
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    # Resize and save
    img = Image.open(in_path).convert("RGB")
    img = img.resize((size, size), Image.LANCZOS)
    img.save(out_path, quality=95)

    return out_path

def process_csv(csv_path, base_dir, output_dir, size):
    df = pd.read_csv(csv_path)
    items = []
    idx = 0

    for _, row in df.iterrows():
        try:
            img_path = process_row(row, base_dir, output_dir, size, idx)
        except:
            continue

        caption = create_caption(row["style"], row["room_type"])
        items.append({"file_path": img_path, "text": caption})
        idx += 1

    return items


def main():
    base_dir = "/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata"  # CHANGE THIS
    output_dir = "/content/processed"                     # output
    size = 512                                            # or 1024

    os.makedirs(output_dir, exist_ok=True)

    train_items = process_csv("/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata/train_data.csv", base_dir, output_dir, size)
    test_items = process_csv("/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata/test_data.csv", base_dir, output_dir, size)

    all_items = train_items + test_items
    random.shuffle(all_items)

    val_count = int(len(all_items) * 0.05)
    val_items = all_items[:val_count]
    train_items = all_items[val_count:]

    with open(os.path.join(output_dir, "train_captions.jsonl"), "w") as f:
        for item in train_items:
            f.write(json.dumps(item) + "\n")

    with open(os.path.join(output_dir, "val_captions.jsonl"), "w") as f:
        for item in val_items:
            f.write(json.dumps(item) + "\n")

    print("Done! Processed images saved to:", output_dir)

main()


Done! Processed images saved to: /content/processed


In [9]:
!pip install -q diffusers accelerate transformers safetensors torchvision


In [10]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora_sdxl.py -O train_lora_sdxl.py


--2025-11-14 20:02:04--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora_sdxl.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56285 (55K) [text/plain]
Saving to: ‘train_lora_sdxl.py’

train_lora_sdxl.py  100%[===================>]  54.97K  --.-KB/s    in 0s      

2025-11-14 20:02:05 (114 MB/s) - ‘train_lora_sdxl.py’ saved [56285/56285]



In [11]:
!accelerate launch train_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --dataset_config="{'train_data_dir':'/content/processed', 'caption_column':'text', 'image_column':'file_path'}" \
  --validation_prompt="Modern living room with warm lighting and wooden flooring" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --max_train_steps=2000 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --mixed_precision="fp16" \
  --output_dir="/content/lora_output"


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-14 20:03:42.238915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763150622.682983   12890 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763150622.795829   12890 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763150622.935641   12890 computation_plac